In [1]:
import pygame
import math
import random

pygame.init()

WIDTH, HEIGHT = 600, 400
screen = pygame.display.set_mode((WIDTH, HEIGHT), pygame.RESIZABLE)
pygame.display.set_caption("catch the jellyfishes, avoid Plankton!")

#музыка
pygame.mixer.init()
pygame.mixer.music.load("atmosphere-of-the-night-sample.wav")
pygame.mixer.music.play(-1) 

score_sound = pygame.mixer.Sound("water-bubbles-4.mp3") 
score_sound_ghost = pygame.mixer.Sound("female-ghost-laughing-horror-scary-ghost-crawling.mp3") 
damage_sound = pygame.mixer.Sound("spongebob-disappointed.mp3")  # звук при получении урона
plankton_think_sound = pygame.mixer.Sound("Voicy_Plankton Think.mp3")  #звук планктона при появлении

# Функция для удаления фона у изображений
def remove_background(image_path, background_color=(0, 0, 0)):
    try:
        image = pygame.image.load(image_path).convert()
        image.set_colorkey(background_color)
        return image
    except:
        #если не удалось загрузить то обычное изображение
        return pygame.image.load(image_path).convert_alpha()

# Функции для масштабирования координат
def scale_x(x):
    return x * WIDTH / 600

def scale_y(y):
    return y * HEIGHT / 400

def inverse_scale_x(x):
    """Преобразует координату X из текущего размера в базовый (600)"""
    return x * 600 / WIDTH

def inverse_scale_y(y):
    """Преобразует координату Y из текущего размера в базовый (400)"""
    return y * 400 / HEIGHT

#изображения
background_original = pygame.image.load("photo_2025-11-13_21-00-52.jpg")
ball_image_original = remove_background("star.jpg", (0, 0, 0))
gun_image_original = remove_background("patric.jpg", (0, 0, 0))
plankton_image_original = remove_background("plancton.jpg", (0, 0, 0))
target_images_original = []
image_paths = [
    "meduse2jpg.jpg",  #0 цель
    "meduse2jpg.jpg",  #1 цель  
    "meduse1jpg.jpg",  #2 цель
    "meduse1jpg.jpg",  #3 цель
    "meduse2jpg.jpg",  #4 цель
    "pirat.jpg"  #5 цель
]

for path in image_paths:
    img = remove_background(path, (0, 0, 0)) #применяем функцию удаления фона
    target_images_original.append(img)

#масштабируем изображения под текущий размер
def scale_images():
    global background, ball_image, gun_image, plankton_image, target_images
    global gun_width, gun_height, plankton_width, plankton_height, targets_width, targets_height
    background = pygame.transform.scale(background_original, (WIDTH, HEIGHT))
    ball_image = pygame.transform.scale(ball_image_original, (int(scale_x(40)), int(scale_y(30))))
    gun_image = pygame.transform.scale(gun_image_original, (int(scale_x(70)), int(scale_y(100))))
    gun_width = int(scale_x(70))  #определяем gun_width
    gun_height = int(scale_y(100))  #определяем gun_height
    plankton_image = pygame.transform.scale(plankton_image_original, (int(scale_x(40)), int(scale_y(60))))
    plankton_width = int(scale_x(40))  #определяем plankton_width
    plankton_height = int(scale_y(60))  #определяем plankton_height
    
    target_images = []
    for i in range(6):
        scaled_img = pygame.transform.scale(target_images_original[i], 
                                           (int(scale_x(40)), int(scale_y(50))))
        target_images.append(scaled_img)
    targets_width = int(scale_x(40))  
    targets_height = int(scale_y(50))  

#вводим переменные когда уже посчитали их значения относительно экрана, начальные координаты
gun_width = int(scale_x(70))  
gun_height = int(scale_y(100))  
plankton_width = int(scale_x(40)) 
plankton_height = int(scale_y(60))  
targets_width = int(scale_x(40))  
targets_height = int(scale_y(50))  
scale_images()

#пушка. инициализация параметров (в относительных координатах)
center_x_ratio, ground_y_ratio = 0.5, 0.75  #относ координаты (0-1)
gun_x, gun_y = WIDTH * center_x_ratio, HEIGHT * ground_y_ratio
gun_width_ratio, gun_height_ratio = 70/600, 100/400 

#мяч
ball_radius_ratio = 15/400  # Относительный радиус
x, y = gun_x, gun_y
radius = HEIGHT * ball_radius_ratio
vx, vy = 0, 0
gravity_ratio = 0.5/400  # Относительная гравитация
gravity = HEIGHT * gravity_ratio
power_ratio = 12/600  # Относительная мощность
power = WIDTH * power_ratio
angle = 45
bounce_loss = 0.7
on_ground = True

#параметры для прыжка патрика
gun_gravity_ratio = 0.8/400 
gun_gravity = HEIGHT * gun_gravity_ratio
gun_y_speed = 0
gun_jump_power_ratio = -12/400
gun_jump_power = HEIGHT * gun_jump_power_ratio

#переменные для бесконечного фона
bg_x = 0
bg_speed = 0  #скорость смещения фона

#параметры для планктона (в относительных координатах)
plankton_width_ratio, plankton_height_ratio = 40/600, 60/400
plankton_x = WIDTH  #начинает справа за экраном
plankton_y = random.randint(int(HEIGHT * 0.5), int(HEIGHT * 0.8)) #gланктон спавнится в нижней части экрана
plankton_speed_ratio = -3/600 
plankton_speed = WIDTH * plankton_speed_ratio
plankton_active = False
plankton_spawn_timer = 0
plankton_spawn_interval = 10000  #10 сек
plankton_last_spawn = 0
plankton_damage_cooldown = 0  #задержка между получением урона от планктона
plankton_sound_played = False  #флаг, проверяющий чтобы звук воспроизводился только один раз при появлении

#предупреждения о планктоне
warning_duration = 2000  #2 сек
show_warning = False
warning_start_time = 0

#здоровье Патрика
health = 3

#создаём 6 целей
num_targets = 6
#цели распределены по всему миру
world_width_ratio = 3.0  #мир в 3 раза шире экрана
world_width = WIDTH * world_width_ratio
targets_x = [random.randint(int(WIDTH * 0.33), int(world_width - WIDTH * 0.33)) for _ in range(num_targets)]
targets_y = [random.randint(int(HEIGHT * 0.175), int(HEIGHT * 0.45)) for _ in range(num_targets)]
targets_speed_x = [random.uniform(0.5, 2.0) * random.choice([-1, 1]) for _ in range(num_targets)]
targets_speed_y = [random.uniform(0.3, 1.0) * random.choice([-1, 1]) for _ in range(num_targets)]
targets_active = [True] * num_targets

font = pygame.font.Font(None, int(HEIGHT * 0.06))  #шрифт масштабируется с высотой
warning_font = pygame.font.Font(None, int(HEIGHT * 0.08))  #больший шрифт для предупреждения

scored_flag = False
score = 0
total_targets = num_targets

running = True
clock = pygame.time.Clock()

#игровой цикл
while running:
    current_time = pygame.time.get_ticks()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        elif event.type == pygame.VIDEORESIZE:
            #обновляем размеры окна при изменении
            WIDTH, HEIGHT = event.w, event.h
            screen = pygame.display.set_mode((WIDTH, HEIGHT), pygame.RESIZABLE)
            
            #масштабируем все изображения под новый размер
            scale_images()
            
            #обновляем относительные параметры
            gun_x = WIDTH * center_x_ratio
            gun_y = HEIGHT * ground_y_ratio
            radius = HEIGHT * ball_radius_ratio
            gravity = HEIGHT * gravity_ratio
            power = WIDTH * power_ratio
            gun_gravity = HEIGHT * gun_gravity_ratio
            gun_jump_power = HEIGHT * gun_jump_power_ratio
            plankton_speed = WIDTH * plankton_speed_ratio
            world_width = WIDTH * world_width_ratio
            font = pygame.font.Font(None, int(HEIGHT * 0.06))
            warning_font = pygame.font.Font(None, int(HEIGHT * 0.08))

        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                angle = min(angle + 5, 80)
            elif event.key == pygame.K_DOWN:
                angle = max(angle - 5, 10)
            elif event.key == pygame.K_w:
                power = min(power + WIDTH * (1/600), WIDTH * (25/600))
            elif event.key == pygame.K_s:  # Изменение силы броска
                power = max(power - WIDTH * (1/600), WIDTH * (5/600))
            elif event.key == pygame.K_d and on_ground: #стрельба из пушки на D
                rad = math.radians(angle)
                vx = power * math.cos(rad)
                vy = -power * math.sin(rad)
                on_ground = False
                #при броске начинаем движение фона в направлении броска
                bg_speed = vx * 0.5
            elif event.key == pygame.K_LEFT:
                #ручное управление фоном влево - стрелка влево
                bg_speed = WIDTH * (2/600)
            elif event.key == pygame.K_RIGHT:
                #ручное управление фоном вправо - стрелка вправо
                bg_speed = -WIDTH * (2/600)
            elif event.key == pygame.K_SPACE: #прыжок Патрика
                gun_y_speed = gun_jump_power

        elif event.type == pygame.KEYUP:
            if event.key in (pygame.K_LEFT, pygame.K_RIGHT):
                if on_ground:
                    bg_speed = 0  #останавливаем фон, если пушка на земле

    # гравитация для пушки
    gun_y_speed += gun_gravity
    gun_y += gun_y_speed
    
    #проверка земли и потолка для пушки
    if gun_y + gun_height // 2 >= HEIGHT:
        gun_y = HEIGHT - gun_height // 2
        gun_y_speed = 0
    
    if gun_y - gun_height // 2 <= 0:
        gun_y = gun_height // 2
        gun_y_speed = 0

    #обновление смещения фона
    bg_x += bg_speed
    
    # Ограничиваем смещение фона
    if bg_x > world_width - WIDTH:
        bg_x = world_width - WIDTH
    if bg_x < 0:
        bg_x = 0
    
    #мяч движется в мировых координатах
    if not on_ground:
        vy += gravity
        x += vx
        y += vy

    #проверка столкновения мяча с землей
    if y + radius >= HEIGHT:
        y = HEIGHT - radius
        vy = -vy * bounce_loss
        vx *= bounce_loss
        if abs(vy) < 1 and abs(vx) < 1:
            #возвращаем мяч к пушке 
            x, y = gun_x + bg_x, gun_y
            vx, vy = 0, 0
            on_ground = True
            scored_flag = False
            bg_speed = 0

    #обновление позиции целей
    for i in range(num_targets):
        if targets_active[i]:
            targets_x[i] += targets_speed_x[i]
            targets_y[i] += targets_speed_y[i]
            
            #отскок от границ мира
            if targets_x[i] <= 0 or targets_x[i] + targets_width >= world_width:
                targets_speed_x[i] *= -1
            if targets_y[i] <= HEIGHT * 0.125 or targets_y[i] >= HEIGHT * 0.5:
                targets_speed_y[i] *= -1

    #проверка на показ предупреждения о планктоне
    time_until_plankton = plankton_last_spawn + plankton_spawn_interval - current_time
    if not plankton_active and time_until_plankton <= warning_duration and time_until_plankton > 0:
        if not show_warning:
            show_warning = True
            warning_start_time = current_time
    else:
        if show_warning and current_time - warning_start_time > warning_duration:
            show_warning = False

    #спавн планктона каждые 10 секунд
    if current_time - plankton_last_spawn > plankton_spawn_interval and not plankton_active:
        plankton_active = True
        plankton_x = WIDTH  #начинает справа
        plankton_y = random.randint(int(HEIGHT * 0.5), int(HEIGHT * 0.8))
        plankton_last_spawn = current_time
        plankton_sound_played = False  #сбрасываем флаг звука
        show_warning = False  #скрываем предупреждение
    
    #воспроизводитсязвук планктона при появлении 
    if plankton_active and not plankton_sound_played:
        plankton_think_sound.play()
        plankton_sound_played = True
    
    #движение планктона
    if plankton_active:
        plankton_x += plankton_speed
        if plankton_x < -plankton_width:
            plankton_active = False

    #прямоугольник мяча для проверки столкновений
    ball_rect = pygame.Rect(x - radius, y - radius, radius * 2, radius * 2)
    
    #проверка столкновения с активными целями
    for i in range(num_targets):
        if targets_active[i]:
            target_rect = pygame.Rect(targets_x[i], targets_y[i], targets_width, targets_height)
            if ball_rect.colliderect(target_rect) and not scored_flag:
                scored_flag = True
                targets_active[i] = False
                score += 1
                if i < 5: 
                    score_sound.play()
                elif i == 5:
                    score_sound_ghost.play()
            
    #проверка столкновения пушки с планктоном
    gun_rect = pygame.Rect(gun_x - gun_width // 2, gun_y - gun_height // 2, gun_width, gun_height)
    
    if plankton_active and current_time > plankton_damage_cooldown:
        plankton_rect = pygame.Rect(plankton_x, plankton_y, plankton_width, plankton_height)
        
        if gun_rect.colliderect(plankton_rect):
            health -= 1
            damage_sound.play()
            plankton_active = False
            plankton_damage_cooldown = current_time + 1000
            
            if health <= 0:
                #показываем сообщение о конце игры
                game_over_text = font.render("GAME OVER! No health left!", True, (255, 0, 0))
                screen.blit(game_over_text, (WIDTH//2 - game_over_text.get_width()//2, HEIGHT//2))
                pygame.display.flip()
                pygame.time.delay(2000)
                running = False  # Завершаем игру

    #отрисовка бесконечного фона
    bg_offset = bg_x % WIDTH
    screen.blit(background, (bg_offset - WIDTH, 0))
    screen.blit(background, (bg_offset, 0))
    screen.blit(background, (bg_offset + WIDTH, 0))

    #отрисовка траектории
    if on_ground:
        points = []
        rad = math.radians(angle)
        temp_vx = power * math.cos(rad)
        temp_vy = -power * math.sin(rad)
        temp_x, temp_y = gun_x + bg_x, gun_y
        
        for i in range(60):
            temp_vy += gravity
            temp_x += temp_vx
            temp_y += temp_vy
            
            if temp_y + radius >= HEIGHT:
                break
                
            #переводим мировые координаты в экранные
            screen_x = temp_x - bg_x
            screen_y = temp_y
            points.append((int(screen_x), int(screen_y)))
            
        if len(points) > 1:
            pygame.draw.lines(screen, (255, 0, 0), False, points, 2)

    #отрисовка целей
    for i in range(num_targets):
        if targets_active[i]:
            if target_images[i] is not None:
                screen_x = targets_x[i] - bg_x
                if -targets_width < screen_x < WIDTH:
                    screen.blit(target_images[i], (screen_x, targets_y[i]))

    #отрисовка планктона
    if plankton_active:
        pygame.draw.rect(screen, (0, 0, 0), (plankton_x, plankton_y, plankton_width, plankton_height))
        screen.blit(plankton_image, (plankton_x, plankton_y))

    #отрисовка мяча 
    screen_x = x - bg_x
    screen_y = y
    screen.blit(ball_image, (int(screen_x - radius), int(screen_y - radius)))

    #отрисовка пушки
    gun_screen_x = gun_x - gun_width // 2
    gun_screen_y = gun_y - gun_height // 2
    screen.blit(gun_image, (gun_screen_x, gun_screen_y))
    
    #отрисовка текста
    text_angle = font.render(f"angle: {angle}°", True, (255, 0, 0))
    text_power = font.render(f"power: {int(power * 600/WIDTH)}", True, (255, 0, 0)) 
    text_score = font.render(f"score: {score}/{total_targets}", True, (255, 0, 0))
    text_health = font.render(f"health: {health}", True, (0, 255, 0) if health > 1 else (255, 0, 0))
    
    #таймер до появления планктона
    if not plankton_active:
        time_until_plankton = max(0, (plankton_last_spawn + plankton_spawn_interval - current_time) // 1000)
        plankton_timer = font.render(f"Plankton in: {time_until_plankton}s", True, (0, 255, 0))
        screen.blit(plankton_timer, (10, int(HEIGHT * 0.225)))
    
    screen.blit(text_angle, (10, 10))
    screen.blit(text_power, (10, int(HEIGHT * 0.075)))
    screen.blit(text_score, (10, int(HEIGHT * 0.125)))
    screen.blit(text_health, (10, int(HEIGHT * 0.175)))

    #отрисовка предупреждения о планктоне
    if show_warning:
        warning_text = warning_font.render("Берегись Планктона!", True, (255, 0, 0))
        text_width = warning_text.get_width()
        text_height = warning_text.get_height()
        screen.blit(warning_text, (WIDTH//2 - text_width//2, HEIGHT//2 - text_height//2))

    if scored_flag:
        hit_text = font.render("JELLY ON THE HOOK!!!!", True, (0, 255, 0))
        screen.blit(hit_text, (WIDTH//2 - hit_text.get_width()//2, int(HEIGHT * 0.125)))
    
    #проверяем победу
    if score >= total_targets:
        victory_text = font.render("VICTORY!", True, (0, 255, 0))
        screen.blit(victory_text, (WIDTH//2 - victory_text.get_width()//2, HEIGHT//2))

    pygame.display.flip()
    clock.tick(60)

pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
